# Подключение зависимостей

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import ccxt
from IPython.display import clear_output, display
import dill

#sns.set_theme()
%matplotlib inline

In [4]:
import mlflow
from mlflow.tracking import MlflowClient

from utilsforecast import losses
from utilsforecast.plotting import plot_series
from neuralforecast import NeuralForecast
from neuralforecast.models import NLinear, DLinear, KAN, NBEATS, NHITS, TFT, LSTM
from neuralforecast.losses.pytorch import MAE, RMSE, HuberLoss, MQLoss

import optuna

# Подключение к MLFlow

In [5]:
mlflow.set_tracking_uri("http://mlflow:5000")

In [6]:
mlflow.set_experiment("btc-usdt_volatility_kan")

<Experiment: artifact_location='mlflow-artifacts:/4', creation_time=1727724856279, experiment_id='4', last_update_time=1727724856279, lifecycle_stage='active', name='btc-usdt_volatility_kan', tags={}>

# Подготовка данных

In [7]:
# Биржа из которой будут браться данные с помощью CCXT
EXCHANGE = ccxt.okx()
# Инструмент в формате символа для обработки
SYMBOL = "BTC/USDT"
# Таймфрейм свеч
TIMEFRAME = "1h"

In [8]:
from_ts = EXCHANGE.parse8601('2022-01-01 00:00:00')

ohlcv_list = []
ohlcv = EXCHANGE.fetch_ohlcv(symbol=SYMBOL, timeframe=TIMEFRAME, since=from_ts, limit=100)
ohlcv_list.append(ohlcv)

while True:
    from_ts = ohlcv[-1][0]
    new_ohlcv = EXCHANGE.fetch_ohlcv(symbol=SYMBOL, timeframe=TIMEFRAME, since=from_ts, limit=100)
    ohlcv.extend(new_ohlcv)

    print(f"\r{EXCHANGE.iso8601(from_ts)}", end="")
    
    if len(new_ohlcv) <= 1:
    	break

2024-10-01T10:00:00.000Z

In [9]:
ohlcv = pd.DataFrame(ohlcv, columns=["date", "open", "high", "low", "close", "volume"])
ohlcv["date"] = ohlcv["date"].apply(lambda x: datetime.datetime.fromtimestamp(x / 1000))
ohlcv["close_pct_change"] = ohlcv["close"].pct_change() * 100
ohlcv['close_pct_change'] = ohlcv['close_pct_change'].fillna(0)
ohlcv.reset_index(drop=True)

,date,open,high,low,close,volume,close_pct_change
0,2022-01-01 00:00:00,46218.3,46742.0,46216.2,46654.3,505.065361,0.000000
1,2022-01-01 01:00:00,46655.1,46943.0,46578.5,46780.1,394.933309,0.269643
2,2022-01-01 02:00:00,46780.1,46927.3,46725.5,46803.7,237.989272,0.050449
3,2022-01-01 03:00:00,46809.8,46902.7,46760.1,46809.1,214.709120,0.011538
4,2022-01-01 04:00:00,46807.7,46879.0,46631.8,46708.5,311.885979,-0.214915
...,...,...,...,...,...,...,...
24346,2024-10-01 07:00:00,64030.9,64128.0,63918.2,64044.9,296.612599,0.022489
24347,2024-10-01 08:00:00,64045.0,64097.9,63824.1,63946.6,259.512373,-0.153486
24348,2024-10-01 09:00:00,63946.5,64102.2,63852.3,63863.9,168.794288,-0.129327
24349,2024-10-01 10:00:00,63864.0,63913.8,63722.0,63899.9,110.108207,0.056370


In [10]:
ohlcv

,date,open,high,low,close,volume,close_pct_change
0,2022-01-01 00:00:00,46218.3,46742.0,46216.2,46654.3,505.065361,0.000000
1,2022-01-01 01:00:00,46655.1,46943.0,46578.5,46780.1,394.933309,0.269643
2,2022-01-01 02:00:00,46780.1,46927.3,46725.5,46803.7,237.989272,0.050449
3,2022-01-01 03:00:00,46809.8,46902.7,46760.1,46809.1,214.709120,0.011538
4,2022-01-01 04:00:00,46807.7,46879.0,46631.8,46708.5,311.885979,-0.214915
...,...,...,...,...,...,...,...
24346,2024-10-01 07:00:00,64030.9,64128.0,63918.2,64044.9,296.612599,0.022489
24347,2024-10-01 08:00:00,64045.0,64097.9,63824.1,63946.6,259.512373,-0.153486
24348,2024-10-01 09:00:00,63946.5,64102.2,63852.3,63863.9,168.794288,-0.129327
24349,2024-10-01 10:00:00,63864.0,63913.8,63722.0,63899.9,110.108207,0.056370


In [11]:
ohlcv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24351 entries, 0 to 24350
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   date              24351 non-null  datetime64[ns]
 1   open              24351 non-null  float64       
 2   high              24351 non-null  float64       
 3   low               24351 non-null  float64       
 4   close             24351 non-null  float64       
 5   volume            24351 non-null  float64       
 6   close_pct_change  24351 non-null  float64       
dtypes: datetime64[ns](1), float64(6)
memory usage: 1.3 MB


In [12]:
ohlcv.shape

(24351, 7)

In [13]:
ohlcv.nunique()

date                24107
open                22916
high                22343
low                 22333
close               22881
volume              24098
close_pct_change    24085
dtype: int64

In [14]:
ohlcv["date"].astype(int)

0        1640995200000000000
1        1640998800000000000
2        1641002400000000000
3        1641006000000000000
4        1641009600000000000
                ...         
24346    1727766000000000000
24347    1727769600000000000
24348    1727773200000000000
24349    1727776800000000000
24350    1727776800000000000
Name: date, Length: 24351, dtype: int64

В данных могут появится дубликаты, от них надо избавлятся

In [15]:
ohlcv = ohlcv.drop_duplicates(subset=['date'], keep='last')

In [16]:
ohlcv

,date,open,high,low,close,volume,close_pct_change
0,2022-01-01 00:00:00,46218.3,46742.0,46216.2,46654.3,505.065361,0.000000
1,2022-01-01 01:00:00,46655.1,46943.0,46578.5,46780.1,394.933309,0.269643
2,2022-01-01 02:00:00,46780.1,46927.3,46725.5,46803.7,237.989272,0.050449
3,2022-01-01 03:00:00,46809.8,46902.7,46760.1,46809.1,214.709120,0.011538
4,2022-01-01 04:00:00,46807.7,46879.0,46631.8,46708.5,311.885979,-0.214915
...,...,...,...,...,...,...,...
24345,2024-10-01 06:00:00,63748.0,64097.9,63744.0,64030.5,240.639318,0.443309
24346,2024-10-01 07:00:00,64030.9,64128.0,63918.2,64044.9,296.612599,0.022489
24347,2024-10-01 08:00:00,64045.0,64097.9,63824.1,63946.6,259.512373,-0.153486
24348,2024-10-01 09:00:00,63946.5,64102.2,63852.3,63863.9,168.794288,-0.129327


In [17]:
def create_traind_df(dots_count=1000):
    train_df = pd.DataFrame(
        columns = ["ds", "y", "unique_id"]   
    )
    
    train_df["ds"] = ohlcv["date"].iloc[-(dots_count)-24:-24]
    train_df["y"] = ohlcv["close_pct_change"].iloc[-(dots_count)-24:-24]
    train_df["unique_id"] = 1
    train_df = train_df.reset_index(drop=True)

    return train_df

In [18]:
train_df_20k = create_traind_df(20000)
train_df_15k = create_traind_df(15000)
train_df_10k = create_traind_df(10000)
train_df_7k = create_traind_df(7000)
train_df_6k = create_traind_df(6000)
train_df_5k = create_traind_df(5000)
train_df_2k = create_traind_df(2000)
train_df_1k = create_traind_df(1000)

In [19]:
test_df = pd.DataFrame(
    columns = ["ds", "y", "unique_id"]   
)

test_df["ds"] = ohlcv["date"].iloc[-48:]
test_df["y"] = ohlcv["close_pct_change"].iloc[-48:]
test_df["unique_id"] = 1
test_df = test_df.reset_index(drop=True)

In [20]:
test_df.shape

(48, 3)

In [21]:
train_df_1k.tail()

,ds,y,unique_id
995,2024-09-30 06:00:00,-0.284847,1
996,2024-09-30 07:00:00,0.344876,1
997,2024-09-30 08:00:00,-0.394028,1
998,2024-09-30 09:00:00,-1.193447,1
999,2024-09-30 10:00:00,0.150924,1


In [22]:
train_df_2k.tail()

,ds,y,unique_id
1995,2024-09-30 06:00:00,-0.284847,1
1996,2024-09-30 07:00:00,0.344876,1
1997,2024-09-30 08:00:00,-0.394028,1
1998,2024-09-30 09:00:00,-1.193447,1
1999,2024-09-30 10:00:00,0.150924,1


In [23]:
test_df.head()

,ds,y,unique_id
0,2024-09-29 11:00:00,0.188332,1
1,2024-09-29 12:00:00,-0.110808,1
2,2024-09-29 13:00:00,0.289061,1
3,2024-09-29 14:00:00,-0.188556,1
4,2024-09-29 15:00:00,0.097881,1


In [24]:
test_df[24:].tail()

,ds,y,unique_id
43,2024-10-01 06:00:00,0.443309,1
44,2024-10-01 07:00:00,0.022489,1
45,2024-10-01 08:00:00,-0.153486,1
46,2024-10-01 09:00:00,-0.129327,1
47,2024-10-01 10:00:00,0.000000,1


# Оптимизация гиперпараметров

In [34]:
def objective(trial):
    try:
        data_set_size = trial.suggest_categorical("data_set_size", ['1k', '2k', '5k', '6k', '7k', '10k', '15k', '20k'])
        
        input_size = trial.suggest_int("input_size", 100, 1000)
        max_steps = trial.suggest_int("max_steps", 100, 2000)
        #max_steps = 1
        batch_size = trial.suggest_int("batch_size", 4, 256)
        windows_batch_size = trial.suggest_int("windows_batch_size", 256, 512)
        #windows_batch_size = 512
        learning_rate = trial.suggest_float("learning_rate", 0.0001, 0.02)

        spline_order = trial.suggest_int("spline_order", 2, 7)
        scale_noise = trial.suggest_float("scale_noise", 0.01, 0.5)
        #scale_noise = 0.1
        scale_base = trial.suggest_float("scale_base", 0.5, 2)
        #scale_base = 1.0
        scale_spline = trial.suggest_float("scale_spline", 0.5, 2)
        #scale_spline = 1.0
        grid_eps = trial.suggest_float("grid_eps", 0.01, 0.1)
        #grid_eps = 0.02
        n_hidden_layers = trial.suggest_int("n_hidden_layers", 1, 3)
        hidden_size = trial.suggest_int("hidden_size", 128, 1024)

        
        models = [KAN(h=48, loss=MQLoss(level=[99, 95, 90, 75, 50]), random_seed=1,
                      input_size=input_size, max_steps=max_steps, 
                      batch_size=batch_size, windows_batch_size=windows_batch_size,
                      learning_rate=learning_rate, spline_order=spline_order,
                      scale_noise=scale_noise, scale_base=scale_base,
                      scale_spline=scale_spline, grid_eps=grid_eps,
                      n_hidden_layers=n_hidden_layers, hidden_size=hidden_size)]
        print(f"models: {models}")

        if data_set_size == "1k":
            train_df = train_df_1k
        elif data_set_size == "2k":
            train_df = train_df_2k
        elif data_set_size == "5k":
            train_df = train_df_5k
        elif data_set_size == "6k":
            train_df = train_df_6k
        elif data_set_size == "7k":
            train_df = train_df_7k
        elif data_set_size == "10k":
            train_df = train_df_10k
        elif data_set_size == "15k":
            train_df = train_df_15k
        elif data_set_size == "20k":
            train_df = train_df_20k
        
        run_name = f'KAN_{str(datetime.datetime.now())}'
        with mlflow.start_run(run_name=run_name) as run:
            # Лучшие параметры полученные в исследовании
            params = {
                "data_set_size": data_set_size,
                "h": 48,
                "loss": "MQLoss(level=[99, 95, 90, 75, 50])",
                "random_seed": 1,
                "input_size": input_size, 
                "max_steps": max_steps, 
                "batch_size": batch_size, 
                "learning_rate": learning_rate, 
                "windows_batch_size": windows_batch_size,
                "spline_order": spline_order,
                "scale_noise": scale_noise,
                "scale_base": scale_base,
                "scale_spline": scale_spline,
                "grid_eps": grid_eps,
                "n_hidden_layers": n_hidden_layers,
                "hidden_size": hidden_size,
            }
            print(f"params: {params}")
            # Сохранение тегов
            mlflow.set_tag("model_name", "KAN")
            # Сохранение параметров
            mlflow.log_params(params)
            
            nf = NeuralForecast(models=models, freq='h')
            nf.fit(train_df)

            # Прогнозирование для test датасета
            forecasts = nf.predict()
            forecasts = forecasts.reset_index()
            current_columns = forecasts.columns.tolist()
            current_columns[2] = current_columns[2].split("-")[0]
            forecasts.columns = current_columns
            
            test_df_metrics = test_df.copy()
            test_df_metrics["KAN"] = forecasts.iloc[:, 2]
            
            # Рассчёт метрик
            rmse = losses.rmse(test_df_metrics.iloc[:24], models=["KAN"])
            mse = losses.mse(test_df_metrics.iloc[:24], models=["KAN"])
            mae = losses.mae(test_df_metrics.iloc[:24], models=["KAN"])
            smape = losses.smape(test_df_metrics.iloc[:24], models=["KAN"])
            
            # Сохранение метрик
            mlflow.log_metric("RMSE", rmse.iloc[0, 1])
            mlflow.log_metric("MSE", mse.iloc[0, 1])
            mlflow.log_metric("MAE", mae.iloc[0, 1])
            mlflow.log_metric("SMAPE", smape.iloc[0, 1])
    
            # Сохранение визуализации
            fig = plot_series(
                pd.concat([train_df.iloc[-300:], test_df[24:]]), 
                forecasts_df=forecasts,
                engine='matplotlib',
                level=[99, 95, 90, 75, 50],
            )
            fig.savefig('forecast.png', bbox_inches='tight')
            plt.close()
            mlflow.log_artifact("forecast.png", "forecast")

            return smape.iloc[0, 1]
        
    except Exception as e:
        print("ОШИБКА: ", e)
        return None
        #raise e

In [35]:
study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler(seed=1))

[I 2024-10-01 11:18:45,557] A new study created in memory with name: no-name-0210b29e-28b9-4c05-aa71-369c979c3a0d


In [ ]:
study.optimize(objective, n_trials=10000)

clear_output()

Seed set to 1


models: [KAN]
params: {'data_set_size': '2k', 'h': 48, 'loss': 'MQLoss(level=[99, 95, 90, 75, 50])', 'random_seed': 1, 'input_size': 457, 'max_steps': 1124, 'batch_size': 110, 'learning_rate': 0.004168599769657198, 'windows_batch_size': 432, 'spline_order': 7, 'scale_noise': 0.02341992066698382, 'scale_base': 1.5057012652676034, 'scale_spline': 1.1259572035506904, 'grid_eps': 0.06028208456011766, 'n_hidden_layers': 1, 'hidden_size': 305}


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name         | Type          | Params | Mode 
-------------------------------------------------------
0 | loss         | MQLoss        | 11     | train
1 | padder_train | ConstantPad1d | 0      | train
2 | scaler       | TemporalNorm  | 0      | train
3 | layers       | ModuleList    | 4.2 M  | train
-------------------------------------------------------
4.2 M     Trainable params
11        Non-trainable params
4.2 M     Total params
16.824    Total estimated model params size (MB)
8         Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]